# Experiments

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [5]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [7]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml")
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [6]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o-mini"
)

View the evaluation results for experiment: 'gpt-4o-mini-41afa336' at:
https://smith.langchain.com/o/5e441d01-8350-44a3-a20d-bcffa2dfe563/datasets/741e2698-1080-4aec-a382-26c36a4c5dca/compare?selectedSessions=75af0c50-f27e-4a57-b743-1c31fb05d831





5it [00:28,  1.93s/it]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What features does LangSmith offer for debugging?,LangSmith offers tracing that provides clear v...,None,LangSmith offers various features for debuggin...,1,2.442297,20f71fb5-bb18-4e73-b648-1385005c6f4b,ba05675a-ab1b-4520-96a4-dbd15e20cb48
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.714595,5652450b-cd81-4e88-a862-20a0fa4827ee,824b5277-1fb0-465e-ad8b-ffc0d23a7369
2,Is there a JavaScript LangSmith SDK?,"Yes, there is a LangSmith SDK for JavaScript (...",None,"Yes, there is a JavaScript LangSmith SDK.",0,1.818601,6d1e2143-2195-4406-b287-115b1d59b66f,4b2307e7-730e-455b-8f79-581b432f0a8a
3,What is the significance of the run_id in Lang...,The context provided does not specify the sign...,None,The run_id in LangSmith is a unique identifier...,1,1.268866,4881524a-e6bc-46f0-ae82-b937e4900290,e908a210-6fd6-466c-9297-ab282b8b86ad
4,Can I integrate LangSmith with other machine l...,The retrieved context does not provide specifi...,None,"Yes, LangSmith can be integrated with various ...",1,1.205265,cc581dcd-99eb-40ac-9b32-f44e37f2a7a8,ace1766f-de8d-41a6-b152-0f4ae8d9d077
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,"To set up tracing to LangSmith with LangChain,...",1,2.609416,9eebc4c8-9930-4ae6-9714-a0a09f0ef0be,c8e31367-6d12-4df2-9ca8-dd9f383cd00a
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.445571,5f53cc5b-b3e7-4625-835c-1c84d7d15297,ef6b6d5b-97bf-402a-a147-68d55db43854
7,Can LangSmith be used for finetuning and model...,"No, LangSmith is primarily focused on the deve...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.853955,9fad8795-f221-475f-ad8e-d8828c23e502,8ca92abb-52e4-4cec-882c-b6850e3180e0
8,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation by ...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.475175,1676c46b-1d93-4c16-9f44-628b0eac2a9e,f705b5be-a890-4598-ae2b-dac613268131
9,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation as p...",None,"Yes, LangSmith supports online evaluation as a...",1,1.581498,e0fb6fae-7aa9-4e3c-bfac-63b41cc6b32a,da75f6d6-e192-49ef-9c59-812524b4908a


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [8]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-731b80f8' at:
https://smith.langchain.com/o/5e441d01-8350-44a3-a20d-bcffa2dfe563/datasets/741e2698-1080-4aec-a382-26c36a4c5dca/compare?selectedSessions=ee2ffca0-c438-4e3c-8353-6007128600df





5it [00:27,  1.84s/it]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What features does LangSmith offer for debugging?,LangSmith offers the ability to quickly unders...,None,LangSmith offers various features for debuggin...,0,1.883816,20f71fb5-bb18-4e73-b648-1385005c6f4b,e0c43ee8-0ab9-45e6-85a4-f46c16dc6cc6
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,2.146299,5652450b-cd81-4e88-a862-20a0fa4827ee,a7501400-9a96-4734-b7c4-3e0654ee38a3
2,Is there a JavaScript LangSmith SDK?,"Yes, there is a LangSmith SDK available for Ja...",None,"Yes, there is a JavaScript LangSmith SDK.",0,1.612587,6d1e2143-2195-4406-b287-115b1d59b66f,f9e20488-2d70-426e-b08f-ffeb830d3f57
3,What is the significance of the run_id in Lang...,The run_id in LangSmith is significant for tra...,None,The run_id in LangSmith is a unique identifier...,1,1.412487,4881524a-e6bc-46f0-ae82-b937e4900290,99ca6eef-5cd0-4a87-a148-5f7a74ab3f3f
4,Can I integrate LangSmith with other machine l...,I don't know the answer to that question based...,None,"Yes, LangSmith can be integrated with various ...",1,1.127426,cc581dcd-99eb-40ac-9b32-f44e37f2a7a8,da1f3136-e9c5-4c0f-a579-8ef14aec1112
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,"To set up tracing to LangSmith with LangChain,...",1,1.535174,9eebc4c8-9930-4ae6-9714-a0a09f0ef0be,89297bfd-6253-4924-b46b-ae0c17a68fef
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",1,1.638478,5f53cc5b-b3e7-4625-835c-1c84d7d15297,816ef200-3542-41bf-a73d-6fa5a6a5f74d
7,Can LangSmith be used for finetuning and model...,LangSmith can be used for refining prompts to ...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.650570,9fad8795-f221-475f-ad8e-d8828c23e502,556f82cc-56d8-4b47-a233-11f7202cdce1
8,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Of...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.249865,1676c46b-1d93-4c16-9f44-628b0eac2a9e,116c3d62-6fd3-473d-b23d-9cd973bf8577
9,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation, whi...",None,"Yes, LangSmith supports online evaluation as a...",0,1.681954,e0fb6fae-7aa9-4e3c-bfac-63b41cc6b32a,a943ec54-2c32-47aa-93e2-9c756fc5a3c5


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [9]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="Initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-8fe83654' at:
https://smith.langchain.com/o/5e441d01-8350-44a3-a20d-bcffa2dfe563/datasets/741e2698-1080-4aec-a382-26c36a4c5dca/compare?selectedSessions=f75d43f0-6587-4d32-bfbc-991d8b4cb6fd





0it [00:21,  2.14s/it]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,3.095382,5652450b-cd81-4e88-a862-20a0fa4827ee,ae1ab9a2-f77e-47db-b276-a3e61e43c239
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents,...",None,"Yes, LangSmith can be used to evaluate agents....",1,2.077614,5f53cc5b-b3e7-4625-835c-1c84d7d15297,42659089-5287-48f1-bebc-35180420e7a6
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for LLM applic...,None,"Yes, LangSmith can be used for fine-tuning and...",0,2.071937,9fad8795-f221-475f-ad8e-d8828c23e502,4b3580a9-40ff-468d-b112-663422f9d9bb
3,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Yo...",None,"Yes, LangSmith supports offline evaluation thr...",1,2.489325,1676c46b-1d93-4c16-9f44-628b0eac2a9e,c66fc8ee-1fb3-4cc8-8af4-0ce351ad7e48
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation as p...",None,"Yes, LangSmith supports online evaluation as a...",1,1.429210,e0fb6fae-7aa9-4e3c-bfac-63b41cc6b32a,b58997f5-b988-4b7a-845a-fd31a69b4779
5,What testing capabilities does LangSmith have?,LangSmith allows developers to create datasets...,None,LangSmith offers capabilities for creating dat...,1,2.047061,e3cd7b9d-329f-4622-af88-d95710f67ac7,6c097278-177d-4604-be94-9c858c611cec
6,What is LangSmith used for in three sentences?,LangSmith is used as a platform for LLM (Large...,None,LangSmith is a platform designed for the devel...,0,2.569823,d6dc92fe-a08f-41aa-a544-08fc22b405b2,9971b0e5-4008-41d9-a96b-ac5e33772ced
7,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can u...",None,You can pass metadata with the @traceable deco...,1,1.715485,f103c769-b5eb-45ab-a3cb-e1699f7530ee,4e4e0d92-df26-45c3-90a2-c16a7aacb5e2
8,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.972576,f08bab51-dc95-4440-ae3b-001bd5573be0,22319901-7339-41fa-bc92-0f642368e56c
9,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,1,1.544218,a561d55c-4191-4f17-aecd-566799fbe722,40f94dd5-02c9-4385-a084-b5c44aa452f1


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [10]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-d02fdea7' at:
https://smith.langchain.com/o/5e441d01-8350-44a3-a20d-bcffa2dfe563/datasets/741e2698-1080-4aec-a382-26c36a4c5dca/compare?selectedSessions=b706158b-aa2f-483e-b1e6-665512fba2fb





it [00:09,  3.26s/it]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What features does LangSmith offer for debugging?,LangSmith offers the ability to rapidly unders...,None,LangSmith offers various features for debuggin...,0,5.734746,20f71fb5-bb18-4e73-b648-1385005c6f4b,1915a7c6-5ee7-49db-86ca-7285b17bc523
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.949710,5652450b-cd81-4e88-a862-20a0fa4827ee,85052b9f-64e2-41fe-94c7-c27dcd9c01de
2,Is there a JavaScript LangSmith SDK?,"Yes, there is a JavaScript LangSmith SDK avail...",None,"Yes, there is a JavaScript LangSmith SDK.",0,1.572442,6d1e2143-2195-4406-b287-115b1d59b66f,9289a896-ccc3-4e2b-8e05-507f2a5a29c3


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [11]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "9eebc4c8-9930-4ae6-9714-a0a09f0ef0be"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="one specific example ids"
)

View the evaluation results for experiment: 'one specific example ids-9bd3af41' at:
https://smith.langchain.com/o/5e441d01-8350-44a3-a20d-bcffa2dfe563/datasets/741e2698-1080-4aec-a382-26c36a4c5dca/compare?selectedSessions=ac76f504-2062-445e-8172-7be91dcef481





it [00:03,  3.14s/it]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,"To set up tracing to LangSmith with LangChain,...",1,2.489901,9eebc4c8-9930-4ae6-9714-a0a09f0ef0be,0b7c101a-0cfe-4fb2-b278-08835e41077b


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [12]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-fc10ca3d' at:
https://smith.langchain.com/o/5e441d01-8350-44a3-a20d-bcffa2dfe563/datasets/741e2698-1080-4aec-a382-26c36a4c5dca/compare?selectedSessions=eb0de8be-8911-4f0d-ae06-df88b0ee164f





0it [00:55,  1.84s/it]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What features does LangSmith offer for debugging?,LangSmith offers tracing functionality that pr...,None,LangSmith offers various features for debuggin...,1,1.950182,20f71fb5-bb18-4e73-b648-1385005c6f4b,4be1ba3b-e5fc-4337-9908-f729159691e7
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.869772,5652450b-cd81-4e88-a862-20a0fa4827ee,a64e0bf3-8bb7-4c04-8a31-7fbcd77d99b3
2,Is there a JavaScript LangSmith SDK?,"Yes, there is a JavaScript LangSmith SDK avail...",None,"Yes, there is a JavaScript LangSmith SDK.",0,1.182489,6d1e2143-2195-4406-b287-115b1d59b66f,cc8cfb11-46df-4209-89a6-8328864590b8
3,What is the significance of the run_id in Lang...,The run_id in LangSmith is used to differentia...,None,The run_id in LangSmith is a unique identifier...,1,1.302883,4881524a-e6bc-46f0-ae82-b937e4900290,785a9ac3-3ccc-4cda-b7be-4c83ec31d6c0
4,Can I integrate LangSmith with other machine l...,"Based on the provided context, there is no spe...",None,"Yes, LangSmith can be integrated with various ...",1,1.978400,cc581dcd-99eb-40ac-9b32-f44e37f2a7a8,9813f0de-9d3a-4958-8719-cd90768487e1
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,"To set up tracing to LangSmith with LangChain,...",1,1.853680,9eebc4c8-9930-4ae6-9714-a0a09f0ef0be,394d6a5a-72af-4da9-8a00-3d43cfd21755
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.638261,5f53cc5b-b3e7-4625-835c-1c84d7d15297,70d7d605-d86c-47b6-b56c-a91787d2d202
7,Can LangSmith be used for finetuning and model...,"LangSmith can be used for prototyping, debuggi...",None,"Yes, LangSmith can be used for fine-tuning and...",0,2.058545,9fad8795-f221-475f-ad8e-d8828c23e502,29494c8c-dd96-4915-8c7d-fe9c729ede42
8,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation by ...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.551524,1676c46b-1d93-4c16-9f44-628b0eac2a9e,fffe41af-06f5-4d00-b505-095edc7ea56f
9,Does LangSmith support online evaluation?,"Yes, LangSmith does support online evaluation ...",None,"Yes, LangSmith supports online evaluation as a...",0,1.913218,e0fb6fae-7aa9-4e3c-bfac-63b41cc6b32a,b1b11c45-87d5-4814-b944-bb5c7f3c32a2


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [13]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-9cfe9f8d' at:
https://smith.langchain.com/o/5e441d01-8350-44a3-a20d-bcffa2dfe563/datasets/741e2698-1080-4aec-a382-26c36a4c5dca/compare?selectedSessions=f11c9e7e-a3a1-4f58-bb25-07c27e8a67c2





5it [00:10,  1.41it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What features does LangSmith offer for debugging?,LangSmith offers the feature of tracing enable...,None,LangSmith offers various features for debuggin...,0,2.018479,20f71fb5-bb18-4e73-b648-1385005c6f4b,7736aae6-8c91-478e-8db9-db594f2ec733
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.089897,5652450b-cd81-4e88-a862-20a0fa4827ee,eff24956-45b2-4f87-9473-d140076b622d
2,Is there a JavaScript LangSmith SDK?,"Yes, there is a LangSmith SDK available for Ja...",None,"Yes, there is a JavaScript LangSmith SDK.",0,2.101682,6d1e2143-2195-4406-b287-115b1d59b66f,708d50d4-1d89-440f-842b-1cbfe615706a
3,Can I integrate LangSmith with other machine l...,I don't have specific information on whether L...,None,"Yes, LangSmith can be integrated with various ...",1,1.318924,cc581dcd-99eb-40ac-9b32-f44e37f2a7a8,11d4858b-eda6-4645-817d-07459ee633be
4,What is the significance of the run_id in Lang...,The run_id in LangSmith is significant for dis...,None,The run_id in LangSmith is a unique identifier...,1,1.560993,4881524a-e6bc-46f0-ae82-b937e4900290,1058bb9c-d7a6-4c44-81ba-d8c269f3bfd9
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,"To set up tracing to LangSmith with LangChain,...",1,2.299769,9eebc4c8-9930-4ae6-9714-a0a09f0ef0be,bc673748-30b7-44eb-bad0-6fc41e12ceac
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.611457,5f53cc5b-b3e7-4625-835c-1c84d7d15297,4fdde2f7-977a-4596-93d0-231722935963
7,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on LLM applicat...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.693558,9fad8795-f221-475f-ad8e-d8828c23e502,3c6ddf09-aa80-4920-afc4-3e8afc51a034
8,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation thr...",None,"Yes, LangSmith supports offline evaluation thr...",1,2.055843,1676c46b-1d93-4c16-9f44-628b0eac2a9e,2fa68c3f-b48d-47ae-8b79-0ad3181ae3b5
9,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation as o...",None,"Yes, LangSmith supports online evaluation as a...",1,1.838777,e0fb6fae-7aa9-4e3c-bfac-63b41cc6b32a,2d79dbd8-d922-4d08-8dfe-afdc00ef1618


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [14]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-7208531d' at:
https://smith.langchain.com/o/5e441d01-8350-44a3-a20d-bcffa2dfe563/datasets/741e2698-1080-4aec-a382-26c36a4c5dca/compare?selectedSessions=19639a8e-d903-43a3-be40-564c88876b69





5it [00:26,  1.77s/it]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What features does LangSmith offer for debugging?,LangSmith offers the ability to have tracing e...,None,LangSmith offers various features for debuggin...,1,2.144091,20f71fb5-bb18-4e73-b648-1385005c6f4b,32b96e39-b520-47db-a7b2-b2606eb88918
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,2.026366,5652450b-cd81-4e88-a862-20a0fa4827ee,8a509661-b321-4698-8fa7-f48baa4c0c04
2,Is there a JavaScript LangSmith SDK?,"Yes, LangSmith has a SDK available for JavaScr...",None,"Yes, there is a JavaScript LangSmith SDK.",0,1.606903,6d1e2143-2195-4406-b287-115b1d59b66f,cfba7891-8698-45e6-a4b6-5379707a34b7
3,What is the significance of the run_id in Lang...,The run_id in LangSmith is significant for dis...,None,The run_id in LangSmith is a unique identifier...,1,1.532764,4881524a-e6bc-46f0-ae82-b937e4900290,19847e34-2cf4-4dc2-b1fb-bddae4f4ef5d
4,Can I integrate LangSmith with other machine l...,I don't know the answer to that question based...,None,"Yes, LangSmith can be integrated with various ...",1,1.724900,cc581dcd-99eb-40ac-9b32-f44e37f2a7a8,16453e70-060f-4099-9266-5b8961a3f1c5
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,"To set up tracing to LangSmith with LangChain,...",1,1.980692,9eebc4c8-9930-4ae6-9714-a0a09f0ef0be,22a34bdb-1022-4008-8859-7ffa64df3512
6,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.511140,5f53cc5b-b3e7-4625-835c-1c84d7d15297,c4575e9a-c178-4e67-86a7-a0ef32303b29
7,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on LLM applicat...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.597537,9fad8795-f221-475f-ad8e-d8828c23e502,7651a888-9962-4746-aac2-d1c8773c8b02
8,Does LangSmith support offline evaluation?,LangSmith supports offline evaluations by allo...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.430387,1676c46b-1d93-4c16-9f44-628b0eac2a9e,43f46631-32a0-45cf-ba20-7c505a6be7d6
9,Does LangSmith support online evaluation?,"Yes, LangSmith does support online evaluation ...",None,"Yes, LangSmith supports online evaluation as a...",0,1.817791,e0fb6fae-7aa9-4e3c-bfac-63b41cc6b32a,2439f140-ad20-4d88-99cd-db0b884944e5
